In [71]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random


  Activating project at `~/code/CT_MPS_mini/CT`


In [72]:
# function conn_pairs(L::Int, ram_phy::Vector{Int})
#     # Generate list of right translations of 1:L more efficiently
#     # Using array comprehension and avoiding modulo operations where possible

#     translations = [circshift(1:L, shift) for shift in 0:L-1]
#     conn_pairs_dict = Dict()
#     for (i1, translation) in enumerate(translations)
#         # println(fold(translation, 0))
#         ram_phy, phy_ram = fold(translation, 0)
#         # println(ram_phy)
#         pairs = []
#         for i in reverse(collect(1:L)[2:end])
#             current_pos = findfirst(x -> x == i, ram_phy)
#             next_pos = findfirst(x -> x == (i-1)%L, ram_phy)
#             pair = (current_pos, next_pos)
#             push!(pairs, pair)
#             # println(i, " is in position $currest_pos carry into $(i-1) which is in position $next_pos")
#         end

#         conn_pairs_dict[i1] = pairs
#     end
#     return conn_pairs_dict
# end

# function fold(translation, ancilla::Int)
#     if ancilla ==0
#         ram_phy = [i for pairs in zip(translation[1:(L÷2)], reverse(translation[(L÷2+1):L])) for i in pairs]
#     elseif ancilla ==1
#         ram_phy = vcat(L+1,[i for pairs in zip(translations[1:(L÷2)], reverse(translations[(L÷2+1):L])) for i in pairs])
#     elseif ancilla ==2
#         error("Not implemented yet")
#     end

#     # phy_ram[physical] = actual in ram
#     phy_ram = fill(0, L+ancilla)
#     for (ram, phy) in enumerate(ram_phy)
#         phy_ram[phy] = ram
#     end

#     return ram_phy, phy_ram
# end


solution one: label the carry indices according to their ram positions and left and right; problem: the boundary tensors are named differently at the turning point

In [73]:
# carry_links = [(Index(2, "Carry,c=$(ram_pos),left"), Index(2, "Carry,c=$(ram_pos),right")) for ram_pos in 1:L]

# gate_vec = []

# for _pair in pairings[i1]
#     if _pair[2] - _pair[1] == 1
#         # forward one
#         T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]][1], carry_links[_pair[1]][2])
#         push!(gate_vec, T_tensor)
#     elseif _pair[2] - _pair[1] == 2
#         # next nearest neighbor case, need to create two qubit gates
#         T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]][1], carry_links[_pair[1]][2])
#         mid_pos = (_pair[1] + _pair[2]) ÷ 2
#         id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[_pair[1]][2], carry_links[mid_pos][2])
#         gate = T_tensor * id_tensor
#         push!(gate_vec, gate)
#     elseif _pair[2] - _pair[1] == -2
#         # next nearest neighbor case, need to create two qubit gates
#         T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]][2], carry_links[_pair[1]][1])
#         mid_pos = (_pair[1] + _pair[2]) ÷ 2
#         id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[_pair[1]][1], carry_links[mid_pos][1])
#         gate = T_tensor * id_tensor
#         push!(gate_vec, gate)
#     elseif _pair[2] - _pair[1] == -1
#         # backward one
#         T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]][2], carry_links[_pair[1]][1])
#         push!(gate_vec, T_tensor)
#     else
#         # more than next nearest neighbor, not implemented yet
#         error("Not implemented yet")
#     end
# end

# # locate the lsb link
# first_forward = pairings[i1][1][2] - pairings[i1][1][1] > 0
# lsb_link_pos = first_forward ? pairings[i1][1][1] : pairings[i1][1][2]
# lsb_link_dir = first_forward ? "left" : "right"
# lsb_link = filterinds(inds(gate_vec[1]), "Carry,c=$(lsb_link_pos),$(lsb_link_dir)")
    
# # include the msb tensor
# msb_pos = pairings[i1][end][2]
# # pairings[i1][end][1]
# mid_pos = (pairings[i1][end][1] + pairings[i1][end][2]) ÷ 2
# msb_link = Index(2, "Carry,msb")
# msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), filterinds(inds(gate_vec[end]), "Carry")[end], msb_link)
# push!(gate_vec, msb_T_tensor)

# # initialize the boundary condition tensor
# lsb_tensor = onehot(lsb_link...=>1)
# # discard the carry out of the msb tensor
# msb_tensor = ITensor(msb_link)
# msb_tensor[msb_link=>1] = 1.0
# msb_tensor[msb_link=>2] = 1.0

# # fixing the boundary condition tensors
# for i in 1:length(gate_vec)
#     gate = gate_vec[i]
#     if msb_link in inds(gate)
#         gate_vec[i] = gate * msb_tensor
#     end
#     if lsb_link... in inds(gate)
#         gate_vec[i] = gate * lsb_tensor
#     end
# end

solution two: name the carry links according to their position without specifying the left or right; this automatically keeps tracks of the connector

In [74]:
# function initialize_gate_vec(pairings_fixed_i1, shift_1_3_bits, qubit_site, ancilla)

#     carry_links = [Index(2, "Carry,c=$(ram_pos-1)") for ram_pos in 1:L+1]

#     gate_vec = ITensor[]

#     for _pair in pairings_fixed_i1
#         if _pair[2] - _pair[1] == 1
#             # forward one
#             T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]], carry_links[_pair[2]])
#             push!(gate_vec, T_tensor)
#         elseif _pair[2] - _pair[1] == 2
#             # next nearest neighbor case, need to create two qubit gates
#             mid_pos = (_pair[1] + _pair[2]) ÷ 2
#             T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]], carry_links[mid_pos])
#             id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[_pair[2]])
#             gate = T_tensor * id_tensor
#             push!(gate_vec, gate)
#         elseif _pair[2] - _pair[1] == -2
#             # next nearest neighbor case, need to create two qubit gates
#             mid_pos = (_pair[1] + _pair[2]) ÷ 2
#             T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]], carry_links[mid_pos])
#             id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[_pair[2]])
#             gate = T_tensor * id_tensor
#             push!(gate_vec, gate)
#         elseif _pair[2] - _pair[1] == -1
#             # backward one
#             T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[_pair[1]], qubit_site[_pair[1]], prime(qubit_site[_pair[1]]), carry_links[_pair[1]], carry_links[_pair[2]])
#             push!(gate_vec, T_tensor)
#         else
#             # more than next nearest neighbor, not implemented yet
#             error("Not implemented yet")
#         end
#     end



#     # locate the lsb link
#     first_forward = pairings[i1][1][2] - pairings[i1][1][1] > 0
#     lsb_link = setdiff(filterinds(inds(gate_vec[1]), "Carry"), filterinds(inds(gate_vec[2]), "Carry"))
#     lsb_tensor = onehot(lsb_link...=>1)

#     # include the msb tensor
#     msb_pos = pairings[i1][end][2]
#     # pairings[i1][end][1]
#     mid_pos = (pairings[i1][end][1] + pairings[i1][end][2]) ÷ 2
#     msb_link = Index(2, "Carry,msb")
#     msb_carry_in = setdiff(filterinds(inds(gate_vec[end]), "Carry"), filterinds(inds(gate_vec[end-1]), "Carry"))
#     msb_tensor = ITensor(msb_link)
#     msb_tensor[msb_link=>1] = 1.0
#     msb_tensor[msb_link=>2] = 1.0

#     msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), msb_carry_in..., msb_link)
#     msb_T_tensor = msb_T_tensor * msb_tensor
#     push!(gate_vec, msb_T_tensor)

#     # set least significant bit tensor boundary condition
#     gate_vec[1] = gate_vec[1] * lsb_tensor

#     return gate_vec
# end

# function qubit_site_number(gate; plev = 0)
#     qubit_legs = filterinds(inds(gate), tags = "Qubit,Site", plev = plev)
#     tag_strs = string.(tags.(qubit_legs))
#     numbers = [parse(Int, match(r"n=(\d+)", tag).captures[1]) for tag in tag_strs]
#     return numbers
# end
# function svd_chain(gate::ITensor; eps = 1e-5, maxbonddim = 10)
#     mpo_vec = Vector{ITensor}()
#     sites = filterinds(inds(gate), tags = "Qubit,Site", plev = 0)
#     for i in 1:length(sites) - 1
#         site_to_left = sites[i]
#         site_number = qubit_site_number(gate, plev = 0)[1]
#         # specify the left indices
#         left_indices = isempty(filterinds(gate, "Link")) ? [site_to_left, prime(site_to_left)] : [site_to_left, prime(site_to_left), filterinds(gate, "Link")...]
#         U, S, V = svd(gate, left_indices, lefttags = "Link,l=$site_number"; cutoff = eps)
#         push!(mpo_vec, U)
#         gate = S * V
#         # println(inds(gate))
#     end
#     return mpo_vec, gate
# end


# function find_collapse_pairs(gate_vec_sorted)
#     # first collapse the folded gate 
#     collapse_pairs = []
#     # identify the pairs
#     i = 1
#     while i <= length(gate_vec_sorted)
#         current_group = [i]
#         sites = filterinds(inds(gate_vec_sorted[i]), tags = "Qubit,Site", plev = 0)
        
#         # Keep checking the next gate for overlap with current group
#         j = i + 1
#         while j <= length(gate_vec_sorted)
#             next_sites = filterinds(inds(gate_vec_sorted[j]), tags = "Qubit,Site", plev = 0)
#             # Check if next gate overlaps with any sites in current group
#             if intersect(sites, next_sites) != []
#                 push!(current_group, j)
#                 # Update sites to include sites from the newly added gate
#                 sites = union(sites, next_sites)
#                 j += 1
#             else
#                 break
#             end
#         end
        
#         push!(collapse_pairs, current_group)
#         i = j  # Move to the next ungrouped gate
#     end
#     return collapse_pairs
# end

# function collapse_gate_vec(gate_vec_sorted)
#     collapse_pairs = find_collapse_pairs(gate_vec_sorted)
#     gate_vec_collapsed = ITensor[]
#     for _pair in collapse_pairs
#         if length(_pair) == 1
#             push!(gate_vec_collapsed, gate_vec_sorted[_pair[1]])
#         else
#             push!(gate_vec_collapsed, lower_prime_level(gate_vec_sorted[_pair[1]] * prime(gate_vec_sorted[_pair[2]], "Qubit,Site"), "Qubit,Site"))
#         end
#     end
#     return gate_vec_collapsed
# end

# function create_mpo(gate_vec_collapsed)
#     mpo_vec = ITensor[]
#     tmp = gate_vec_collapsed[1]
#     for num_gate in 1:length(gate_vec_collapsed) - 1
#         # contract the gates
#         tmp = tmp * prime(gate_vec_collapsed[num_gate+1], "Qubit,Site")

#         # handle the prime level indices
#         primed_indices = prime(filterinds(inds(gate_vec_collapsed[num_gate+1]), tags = "Qubit,Site"))
#         for idx in primed_indices
#             tmp = prime(tmp, -1, idx)
#         end

#         # svd
#         U_vec, remainder = svd_chain(tmp; eps = 1e-10, maxbonddim = 1000);
#         for U in U_vec
#             push!(mpo_vec, U)
#         end

#         # propagate the indices to the remainder
#         tmp = remainder
#     end

#     push!(mpo_vec, tmp);

#     return MPO(mpo_vec)
# end
# function adder_mpo_vec(L, i1, ancilla, folded, numerator, denominator)
#     adder_mpo_vec = MPO[]
#     for i1 in 1:L
#         println("constructed the $(i1)-th mpo")
#         shift_bits, shift_amount = fraction_to_binary_shift(numerator, denominator, L)
#         qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
#         pairing_fixed_i1 = conn_pairs(L, ram_phy)[i1]

#         # initialize the gate vector
#         gate_vec = initialize_gate_vec(pairing_fixed_i1, shift_1_3_bits, qubit_site, ancilla)

#         # sort the gate vector according to the qubit site number
#         gate_sites = [qubit_site_number(gate) for gate in gate_vec]
#         gate_vec_sorted = gate_vec[sortperm(gate_sites)];
#         gate_vec_collapsed = collapse_gate_vec(gate_vec_sorted);

#         # create the mpo
#         adder_mpo = create_mpo(gate_vec_collapsed);
#         push!(adder_mpo_vec, adder_mpo)
#     end
#     return adder_mpo_vec
# end


In [75]:
i1 = 8
numerator = 1
denominator = 3
L = 8
ancilla = 0
folded = true
shift_bits, shift_amount = fraction_to_binary_shift(numerator, denominator, L)
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
pairing_fixed_i1 = conn_pairs(L)[i1]


7-element Vector{Any}:
 (4, 6)
 (6, 8)
 (8, 7)
 (7, 5)
 (5, 3)
 (3, 1)
 (1, 2)

In [76]:
pairing_fixed_i1

7-element Vector{Any}:
 (4, 6)
 (6, 8)
 (8, 7)
 (7, 5)
 (5, 3)
 (3, 1)
 (1, 2)

In [77]:
# initialize the gate vector

gate_vec = initialize_gate_vec(pairing_fixed_i1, shift_bits, qubit_site, L)
initialize_gate_vec(pairing_fixed_i1, shift_bits, qubit_site, L)

# sort the gate vector according to the qubit site number
gate_sites = [qubit_site_number(gate) for gate in gate_vec]
gate_vec_sorted = gate_vec[sortperm(gate_sites)];
gate_vec_collapsed = collapse_gate_vec(gate_vec_sorted);

# create the mpo
adder_mpo = create_mpo(gate_vec_collapsed);

In [78]:
adder_mpo

MPO
[1] ((dim=2|id=489|"Qubit,Site,n=1"), (dim=2|id=489|"Qubit,Site,n=1")', (dim=3|id=490|"Link,l=1"))
[2] ((dim=3|id=490|"Link,l=1"), (dim=2|id=461|"Qubit,Site,n=2"), (dim=2|id=461|"Qubit,Site,n=2")', (dim=2|id=258|"Link,l=2"))
[3] ((dim=2|id=258|"Link,l=2"), (dim=2|id=518|"Qubit,Site,n=3"), (dim=2|id=518|"Qubit,Site,n=3")', (dim=2|id=761|"Link,l=3"))
[4] ((dim=2|id=761|"Link,l=3"), (dim=2|id=779|"Qubit,Site,n=4"), (dim=2|id=779|"Qubit,Site,n=4")', (dim=4|id=505|"Link,l=4"))
[5] ((dim=4|id=505|"Link,l=4"), (dim=2|id=943|"Qubit,Site,n=5"), (dim=2|id=943|"Qubit,Site,n=5")', (dim=4|id=444|"Link,l=5"))
[6] ((dim=4|id=444|"Link,l=5"), (dim=2|id=392|"Qubit,Site,n=6"), (dim=2|id=392|"Qubit,Site,n=6")', (dim=4|id=701|"Link,l=6"))
[7] ((dim=4|id=701|"Link,l=6"), (dim=2|id=694|"Qubit,Site,n=7"), (dim=2|id=694|"Qubit,Site,n=7")', (dim=3|id=220|"Link,l=7"))
[8] ((dim=3|id=220|"Link,l=7"), (dim=2|id=342|"Qubit,Site,n=8"), (dim=2|id=342|"Qubit,Site,n=8")')


In [ ]:
carry_links = [(Index(2, "Carry,c=$(ram_pos),left"), Index(2, "Carry,c=$(ram_pos),right")) for ram_pos in 1:L]
T_tensor_vec = []
for i in 1:length(pairings[i1])
    pair = pairings[i1][i]
    forward = pair[2] - pair[1] > 0
    site_pos = pair[1]

    if forward
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[site_pos], qubit_site[site_pos], prime(qubit_site[site_pos]), carry_links[site_pos][1], carry_links[pair[2]][1])
        push!(T_tensor_vec, T_tensor)
    else
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[site_pos], qubit_site[site_pos], prime(qubit_site[site_pos]), carry_links[site_pos][2], carry_links[pair[2]][2])
        push!(T_tensor_vec, T_tensor)
    end
end

# locate the lsb link
first_forward = pairings[i1][1][2] - pairings[i1][1][1] > 0
lsb_link_pos = first_forward ? pairings[i1][1][1] : pairings[i1][1][2]
lsb_link_dir = first_forward ? "left" : "right"
lsb_link = filterinds(inds(T_tensor_vec[1]), "Carry,c=$(lsb_link_pos),$(lsb_link_dir)")
    
# include the msb tensor
msb_pos = pairings[i1][end][2]
# pairings[i1][end][1]
mid_pos = (pairings[i1][end][1] + pairings[i1][end][2]) ÷ 2
msb_link = Index(2, "Carry,msb")
msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), inds(T_tensor_vec[end])[end], msb_link)
push!(T_tensor_vec, msb_T_tensor)

# initialize the boundary condition tensor
lsb_tensor = onehot(lsb_link...=>1)
# discard the carry out of the msb tensor
msb_tensor = ITensor(msb_link)
msb_tensor[msb_link=>1] = 1.0
msb_tensor[msb_link=>2] = 1.0

# fixing the boundary condition tensors
for i in 1:length(T_tensor_vec)
    T_tensor = T_tensor_vec[i]
    if msb_link in inds(T_tensor)
        T_tensor_vec[i] = T_tensor * msb_tensor
    end
    if lsb_link... in inds(T_tensor)
        T_tensor_vec[i] = T_tensor * lsb_tensor
    end
end

In [18]:
length(gate_vec)

5

In [17]:
for gate in gate_vec
    println(inds(gate))
end


((dim=2|id=959|"Qubit,Site,n=2"), (dim=2|id=959|"Qubit,Site,n=2")', (dim=2|id=318|"Carry,c=2,left"), (dim=2|id=182|"Carry,c=2,right"), (dim=2|id=108|"Carry,c=3,right"), (dim=2|id=375|"Qubit,Site,n=3"), (dim=2|id=375|"Qubit,Site,n=3")', (dim=2|id=978|"Carry,c=3,left"))
((dim=2|id=608|"Qubit,Site,n=4"), (dim=2|id=608|"Qubit,Site,n=4")', (dim=2|id=476|"Carry,c=4,left"), (dim=2|id=299|"Carry,c=4,right"), (dim=2|id=581|"Carry,c=5,right"), (dim=2|id=213|"Qubit,Site,n=5"), (dim=2|id=213|"Qubit,Site,n=5")', (dim=2|id=734|"Carry,c=5,left"))
((dim=2|id=850|"Qubit,Site,n=6"), (dim=2|id=850|"Qubit,Site,n=6")', (dim=2|id=731|"Carry,c=6,left"), (dim=2|id=606|"Carry,c=6,right"), (dim=2|id=100|"Carry,c=7,right"), (dim=2|id=895|"Qubit,Site,n=7"), (dim=2|id=895|"Qubit,Site,n=7")', (dim=2|id=492|"Carry,c=7,left"))
((dim=2|id=895|"Qubit,Site,n=7"), (dim=2|id=895|"Qubit,Site,n=7")', (dim=2|id=100|"Carry,c=7,right"), (dim=2|id=492|"Carry,c=7,left"), (dim=2|id=731|"Carry,c=6,left"), (dim=2|id=850|"Qubit,Site

In [31]:
inds_tracker = []
tmp_tensor = T_tensor_vec[1]
for i in 1:length(T_tensor_vec)-1
    tmp_tensor = tmp_tensor * T_tensor_vec[i+1]
    site_indices = filterinds(inds(tmp_tensor), "Qubit")
    U, S, V = svd(tmp_tensor, site_indices)
    push!(inds_tracker, inds(U))
    tmp_tensor = S * V
    println(inds(U))
    println(inds(S * V))
    println("")
end


In [33]:
for indices in inds_tracker
    println(length(indices))
    println(indices)
end

5
((dim=2|id=473|"Qubit,Site,n=2"), (dim=2|id=473|"Qubit,Site,n=2")', (dim=2|id=553|"Qubit,Site,n=4"), (dim=2|id=553|"Qubit,Site,n=4")', (dim=2|id=60|"Link,u"))
3
((dim=2|id=461|"Qubit,Site,n=6"), (dim=2|id=461|"Qubit,Site,n=6")', (dim=4|id=393|"Link,u"))
3
((dim=2|id=236|"Qubit,Site,n=8"), (dim=2|id=236|"Qubit,Site,n=8")', (dim=4|id=97|"Link,u"))
3
((dim=2|id=981|"Qubit,Site,n=7"), (dim=2|id=981|"Qubit,Site,n=7")', (dim=4|id=864|"Link,u"))
3
((dim=2|id=596|"Qubit,Site,n=5"), (dim=2|id=596|"Qubit,Site,n=5")', (dim=4|id=861|"Link,u"))
3
((dim=2|id=330|"Qubit,Site,n=3"), (dim=2|id=330|"Qubit,Site,n=3")', (dim=4|id=944|"Link,u"))
3
((dim=2|id=490|"Qubit,Site,n=1"), (dim=2|id=490|"Qubit,Site,n=1")', (dim=4|id=577|"Link,u"))


In [ ]:

T_tensor_vec = []
for i in 1:length(pairings[i1])
    pair = pairings[i1][i]
    forward = pair[2] - pair[1] > 0

    if forward
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[pair[1]], qubit_site[pair[1]], prime(qubit_site[pair[1]]), carry_links[pair[1]], carry_links[pair[2]])
        push!(T_tensor_vec, T_tensor)
    else
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[pair[1]], qubit_site[pair[1]], prime(qubit_site[pair[1]]), carry_links[pair[1]], carry_links[pair[2]])
        push!(T_tensor_vec, T_tensor)
    end

# include the msb tensor
msb_pos = pairings[i1][end][2]
# pairings[i1][end][1]
mid_pos = (pairings[i1][end][1] + pairings[i1][end][2]) ÷ 2
msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), carry_links[msb_pos], carry_links[mid_pos])
push!(T_tensor_vec, msb_T_tensor)

ErrorException: syntax: incomplete: "for" at /Users/youtao/code/CT_MPS_mini/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:2 requires end

In [45]:
for i in 1:length(T_tensor_vec)
    println(inds(T_tensor_vec[i]))
end


In [30]:
for i in 1:length(T_tensor_vec)
    println(inds(T_tensor_vec[i]))
end


((dim=2|id=403|"Qubit,Site,n=2"), (dim=2|id=403|"Qubit,Site,n=2")', (dim=2|id=382|"Carry,c=1"), (dim=2|id=451|"Carry,c=3"))
((dim=2|id=939|"Qubit,Site,n=4"), (dim=2|id=939|"Qubit,Site,n=4")', (dim=2|id=451|"Carry,c=3"), (dim=2|id=148|"Carry,c=5"))
((dim=2|id=679|"Qubit,Site,n=6"), (dim=2|id=679|"Qubit,Site,n=6")', (dim=2|id=148|"Carry,c=5"), (dim=2|id=506|"Carry,c=7"))
((dim=2|id=601|"Qubit,Site,n=8"), (dim=2|id=601|"Qubit,Site,n=8")', (dim=2|id=506|"Carry,c=7"), (dim=2|id=675|"Carry,c=6"))
((dim=2|id=234|"Qubit,Site,n=7"), (dim=2|id=234|"Qubit,Site,n=7")', (dim=2|id=675|"Carry,c=6"), (dim=2|id=288|"Carry,c=4"))
((dim=2|id=705|"Qubit,Site,n=5"), (dim=2|id=705|"Qubit,Site,n=5")', (dim=2|id=288|"Carry,c=4"), (dim=2|id=720|"Carry,c=2"))
((dim=2|id=620|"Qubit,Site,n=3"), (dim=2|id=620|"Qubit,Site,n=3")', (dim=2|id=720|"Carry,c=2"), (dim=2|id=712|"Carry,c=0"))


In [11]:
# initialize the boundary condition tensor
lsb_tensor = onehot(carry_links[2]=>1)
# discard the carry out of the msb tensor
msb_tensor = ITensor(carry_links[1])
msb_tensor[carry_links[1]=>1] = 1.0
msb_tensor[carry_links[1]=>2] = 1.0

# fixing the boundary condition tensors
for i in 1:length(T_tensor_vec)
    T_tensor = T_tensor_vec[i]
    if carry_links[1] in inds(T_tensor)
        T_tensor_vec[i] = T_tensor * msb_tensor
    end
    if carry_links[2] in inds(T_tensor)
        T_tensor_vec[i] = T_tensor * lsb_tensor
    end
end


In [22]:
length(T_tensor_vec)

7

In [17]:
length[mpo_vec]

MethodError: MethodError: no method matching getindex(::typeof(length), ::Vector{Any})

In [ ]:
# need to initialize the gate_vec's according to the pairing first
gate_vec = []
for pair in pairing 
    start_pos = pair[1]
    end_pos = pair[2]
    if end_pos - start_pos == 1
        # forward one 
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[end_pos])
        push!(gate_vec, T_tensor)
        continue
    elseif end_pos - start_pos == 2
        mid_pos = (start_pos + end_pos) ÷ 2
        # next nearest neighbor case, need to create two qubit gates
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[mid_pos])
        id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[end_pos])
        gate = T_tensor * id_tensor
        push!(gate_vec, gate)
    elseif end_pos - start_pos == -2
        mid_pos = (start_pos + end_pos) ÷ 2
        # next nearest neighbor case, need to create two qubit gates
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[mid_pos])
        id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[end_pos])
        gate = T_tensor * id_tensor
        push!(gate_vec, gate)
    elseif end_pos - start_pos == -1
        # backward one
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[end_pos])
        push!(gate_vec, T_tensor)
        continue
    else
        # more than next nearest neighbor, not implemented yet
        error("Not implemented yet")
    end
end

# msb T tensor also needs to be included
lsb = pairing[begin][1];
msb_pos = pairing[end][2];
msb_plus_one = (pairing[end][1] + pairing[end][2]) ÷ 2;
msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), carry_links[msb_pos], carry_links[msb_plus_one]);
push!(gate_vec, msb_T_tensor);

In [129]:
# initialize the boundary condition tensor
lsb_tensor = onehot(carry_links[2]=>1)
# discard the carry out of the msb tensor
msb_tensor = ITensor(carry_links[1])
msb_tensor[carry_links[1]=>1] = 1.0
msb_tensor[carry_links[1]=>2] = 1.0

1.0

Actually i think maybe i dont need the id_tensor_4d at all; i can direct contract them according to the pairing sequence

In [ ]:
# fixing the boundary condition tensors
for igate in 1:length(gate_vec)
    gate = gate_vec[igate]
    if carry_links[2] in inds(gate)
        gate_vec[igate] = gate * msb_tensor
    end
    if carry_links[1] in inds(gate)
        gate_vec[igate] = gate * lsb_tensor
    end
end


In [131]:
for gate in gate_vec
    println(inds(gate))
end


((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=78|"Carry,c=3"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"))
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=295|"Carry,c=6"))
((dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=295|"Carry,c=6"), (dim=2|id=695|"Carry,c=4"), (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")')
((dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")', (dim=2|id=695|"Carry

In [132]:
for idx in 1:length(gate_vec)-1
    println(inds(gate_vec[idx] * gate_vec[idx+1]))
end

((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"), (dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")', (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=295|"Carry,c=6"))
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")',

In [133]:
length(gate_vec)

8

In [134]:
for igate in 1:length(gate_vec)
    gate = gate_vec[igate]
    pair = pairing[igate]
    println(pair[1], "-->", pair[2])
    println(inds(gate))
end
# for idx in 1:length(gate_vec), find all the gates that has qubit_site[idx] as an index, and put them in a list

2-->4
((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=78|"Carry,c=3"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"))
4-->6
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
6-->8
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
8-->7
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=295|"Carry,c=6"))
7-->5
((dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=295|"Carry,c=6"), (dim=2|id=695|"Carry,c=4"), (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")')
5-->3
((dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qub

LoadError: BoundsError: attempt to access 7-element Vector{Any} at index [8]

In [ ]:
# for idx in 1:length(gate_vec), find all the gates that has qubit_site[idx] as an index, and put them in a list
# then contract all the gates in the list together
# then svd according to the qubit_site index 

In [ ]:
# Example: find intersection between two sets a and b
a = Set([1, 2, 3, 4])
b = Set([3, 4, 5, 6])
intersection = intersect(a, b)
println("Intersection: ", intersection)


Intersection: Set([4, 3])


In [ ]:
intersection = intersect(Set(inds(gate_vec[1])), Set(inds(gate_vec[2])))

Set{Index{Int64}} with 1 element:
  (dim=2|id=497|"Carry,c=3")

In [ ]:
commoninds(gate_vec[1], gate_vec[2])

1-element Vector{Index{Int64}}:
 (dim=2|id=497|"Carry,c=3")